In [34]:
import os
import pandas as pd
import numpy as np

In [35]:
from autokeras.keras_layers import ExpandLastDim
from autokeras.keras_layers import CastToFloat32
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, LSTM, SimpleRNN
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.layers.experimental.preprocessing import RandomTranslation, RandomFlip
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.applications import EfficientNetB7
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

# Make Input

In [36]:
def calculateWeight(tlx):
    tlx_weight = {'Mental':[0], 
                  'Physical':[0], 
                  'Temporal':[0], 
                  'Effort':[0],
                  'Performance':[0],
                  'Frustration':[0],
                  'Sum':[0]}
    tlx_weight = pd.DataFrame(tlx_weight)
    for i in range(len(tlx)):
        score = [0,0,0,0,0,0,0]
        for col1 in range(1,len(tlx.columns)):
            for col2 in range(col1+1, len(tlx.columns)):
                if tlx[tlx.columns[col1]][i] > tlx[tlx.columns[col2]][i]:
                    score[col1-1]+=1
                elif tlx[tlx.columns[col1]][i] < tlx[tlx.columns[col2]][i]:
                    score[col2-1]+=1
                else :
                    score[col1-1]+=0.5
                    score[col2-1]+=0.5
                    
        score[6] = score[0]+score[1]+score[2]+score[3]+score[4]+score[5]
        tlx_weight.loc[i]=score
    #print(tlx_weight.loc[0])
    return tlx_weight

def calculate_tlxLevel(tlx, tlx_weight):
    result = {'Mental':[0], 
                  'Physical':[0], 
                  'Temporal':[0], 
                  'Effort':[0],
                  'Performance':[0],
                  'Frustration':[0],
                  'Score':[0]}
    result = pd.DataFrame(result)
    for i in range(len(tlx)):
        score = [0,0,0,0,0,0,0]
        for col in range(len(tlx_weight.columns)-1):
            score[col] = int(tlx[tlx.columns[col+1]].loc[i] * tlx_weight[tlx_weight.columns[col]].loc[i] )
        score[6] =int((score[0]+score[1]+score[2]+score[3]+score[4]+score[5] )/ tlx_weight[tlx_weight.columns[6]].loc[i]/10 + 0.5)
        if score[6]>10: score[6]=10
        if score[6]<0: score[6]=0
        result.loc[i]=score
    return result['Score']

In [38]:
def split_fft_data(df, label, end):
    col = ['POW.AF3.Theta', 'POW.AF3.Alpha',
       'POW.AF3.BetaL', 'POW.AF3.BetaH', 'POW.AF3.Gamma', 'POW.F7.Theta',
       'POW.F7.Alpha', 'POW.F7.BetaL', 'POW.F7.BetaH', 'POW.F7.Gamma',
       'POW.F3.Theta', 'POW.F3.Alpha', 'POW.F3.BetaL', 'POW.F3.BetaH',
       'POW.F3.Gamma', 'POW.FC5.Theta', 'POW.FC5.Alpha', 'POW.FC5.BetaL',
       'POW.FC5.BetaH', 'POW.FC5.Gamma', 'POW.T7.Theta', 'POW.T7.Alpha',
       'POW.T7.BetaL', 'POW.T7.BetaH', 'POW.T7.Gamma', 'POW.P7.Theta',
       'POW.P7.Alpha', 'POW.P7.BetaL', 'POW.P7.BetaH', 'POW.P7.Gamma',
       'POW.O1.Theta', 'POW.O1.Alpha', 'POW.O1.BetaL', 'POW.O1.BetaH',
       'POW.O1.Gamma', 'POW.O2.Theta', 'POW.O2.Alpha', 'POW.O2.BetaL',
       'POW.O2.BetaH', 'POW.O2.Gamma', 'POW.P8.Theta', 'POW.P8.Alpha',
       'POW.P8.BetaL', 'POW.P8.BetaH', 'POW.P8.Gamma', 'POW.T8.Theta',
       'POW.T8.Alpha', 'POW.T8.BetaL', 'POW.T8.BetaH', 'POW.T8.Gamma',
       'POW.FC6.Theta', 'POW.FC6.Alpha', 'POW.FC6.BetaL', 'POW.FC6.BetaH',
       'POW.FC6.Gamma', 'POW.F4.Theta', 'POW.F4.Alpha', 'POW.F4.BetaL',
       'POW.F4.BetaH', 'POW.F4.Gamma', 'POW.F8.Theta', 'POW.F8.Alpha',
       'POW.F8.BetaL', 'POW.F8.BetaH', 'POW.F8.Gamma', 'POW.AF4.Theta',
       'POW.AF4.Alpha', 'POW.AF4.BetaL', 'POW.AF4.BetaH', 'POW.AF4.Gamma', 'MarkerValueInt']
    col_rename = ['AF3.Theta', 'AF3.Alpha','AF3.BetaL', 'AF3.BetaH', 'AF3.Gamma', 
                  'F7.Theta','F7.Alpha', 'F7.BetaL', 'F7.BetaH', 'F7.Gamma',
                  'F3.Theta', 'F3.Alpha', 'F3.BetaL', 'F3.BetaH','F3.Gamma', 
                  'FC5.Theta', 'FC5.Alpha', 'FC5.BetaL','FC5.BetaH', 'FC5.Gamma', 
                  'T7.Theta', 'T7.Alpha', 'T7.BetaL', 'T7.BetaH', 'T7.Gamma', 
                  'P7.Theta', 'P7.Alpha', 'P7.BetaL', 'P7.BetaH', 'P7.Gamma',
                  'O1.Theta', 'O1.Alpha', 'O1.BetaL', 'O1.BetaH','O1.Gamma',
                  'O2.Theta', 'O2.Alpha', 'O2.BetaL','O2.BetaH', 'O2.Gamma',
                  'P8.Theta', 'P8.Alpha', 'P8.BetaL', 'P8.BetaH', 'P8.Gamma', 
                  'T8.Theta','T8.Alpha', 'T8.BetaL', 'T8.BetaH', 'T8.Gamma',
                  'FC6.Theta', 'FC6.Alpha', 'FC6.BetaL', 'FC6.BetaH','FC6.Gamma', 
                  'F4.Theta', 'F4.Alpha', 'F4.BetaL','F4.BetaH', 'F4.Gamma',
                  'F8.Theta', 'F8.Alpha','F8.BetaL', 'F8.BetaH', 'F8.Gamma',
                  'AF4.Theta', 'AF4.Alpha', 'AF4.BetaL', 'AF4.BetaH', 'AF4.Gamma', 'vis_name']
    data_extraction = df[col]
    data_extraction.columns = col_rename
    rest = data_extraction[data_extraction.vis_name == 0].dropna(axis=0)
    survey = data_extraction[data_extraction.vis_name == 100].dropna(axis=0)
    
    vis = data_extraction[data_extraction.vis_name == 1].reset_index(drop=True).dropna(axis=0)
    vis['label'] = label[0]
    vis.drop(['vis_name'], axis=1, inplace=True)
    
    for i in range(2,end):
        df = data_extraction[data_extraction.vis_name == i].reset_index(drop=True)
        df['label'] = label[i-1]
        df.drop(['vis_name'], axis=1, inplace=True)
        df = df.dropna(axis=0)
        vis = pd.concat([vis,df], ignore_index=True, axis=0)
    return rest, survey, vis

In [37]:
src = 'C:/EEG data/User/'
files = os.listdir(src)
tlx=[]
for f in files:
    tlx.append(pd.read_csv(src+f))
src = 'C:/EEG data/prepross/'
datas = os.listdir(src)
workloadLevel = []
for t in tlx:
    workloadLevel.append(calculate_tlxLevel(t, calculateWeight(t)))
eegData=[]
for d in datas:
    eegData.append(pd.read_csv(src+d))

In [39]:
cnt = 0
for eeg, label in zip (eegData, workloadLevel):
    if cnt == 0 :
        rest, survey, vis = split_fft_data(eeg, label,21)
        cnt+=1
        continue
    elif cnt == 6:
        r, s, v = split_fft_data(eeg, label,21)
    else:
        r, s, v = split_fft_data(eeg, label, 22)
    
    rest = pd.concat([rest,r], ignore_index=True, axis=0)
    survey = pd.concat([survey,s], ignore_index=True, axis=0)
    vis = pd.concat([vis,v], ignore_index=True, axis=0)
    cnt+=1

In [40]:
label=vis['label']
label = to_categorical(label,11)
data=vis.drop(['label'],axis=1)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)
data = pd.DataFrame(scaled, columns = data.columns, index=data.index)
data
print(data.shape, label.shape)

(108212, 70) (108212, 11)


# AUTO KERAS

In [41]:
import autokeras as ak
from tensorflow.keras.utils import plot_model
from datetime import datetime

In [42]:
x_train, x_test, y_train, y_test = train_test_split(data,label, train_size=0.6, 
                                                    random_state=True,
                                                    stratify = label)

print(x_train.shape, x_test.shape,y_train.shape, y_test.shape)

(64927, 70) (43285, 70) (64927, 11) (43285, 11)


In [47]:
trials=[50]
for trial in trials:
    clf_ = ak.StructuredDataClassifier(overwrite=True, max_trials=trial)
    clf_.fit(x=x_train, y=y_train, epochs=20)

    predicted_y = clf_.predict(x_test)
    print(predicted_y)
    loss, acc = clf_.evaluate(x_test, y_test)
    print(clf_.evaluate(x_test, y_test))
    print('Loss: %.3f   Accuracy: %.3f' % (loss,acc))

    model = clf_.export_model()
    model.summary()
    plot_model(model, show_shapes=True)
    tmp = int(acc*100)
    print(tmp)
    model.save('model/'+'FFT_ACC__'+str(tmp)+'_try_'+str(trial), save_format='tf')

Trial 50 Complete [00h 01m 06s]
val_accuracy: 0.6862412095069885

Best val_accuracy So Far: 0.7935025691986084
Total elapsed time: 01h 02m 23s
Epoch 1/20
2029/2029 [==============================] - 9s 4ms/step - loss: 1.5322 - accuracy: 0.4160
Epoch 2/20
2029/2029 [==============================] - 8s 4ms/step - loss: 1.1952 - accuracy: 0.5227
Epoch 3/20
2029/2029 [==============================] - 8s 4ms/step - loss: 1.0249 - accuracy: 0.5873
Epoch 4/20
2029/2029 [==============================] - 8s 4ms/step - loss: 0.9055 - accuracy: 0.6350
Epoch 5/20
2029/2029 [==============================] - 8s 4ms/step - loss: 0.8111 - accuracy: 0.6760
Epoch 6/20
2029/2029 [==============================] - 8s 4ms/step - loss: 0.7264 - accuracy: 0.7118
Epoch 7/20
2029/2029 [==============================] - 8s 4ms/step - loss: 0.6544 - accuracy: 0.7435
Epoch 8/20
2029/2029 [==============================] - 8s 4ms/step - loss: 0.5906 - accuracy: 0.7723
Epoch 9/20
2029/2029 [===================